## Solving the Knapsack Problem using `Scipy` and `Pulp`

In [1]:
import pandas as pd
import numpy as np
from pulp import pulp, LpMaximize, LpVariable, get_solver, LpStatus, lpSum, LpBinary
from scipy.optimize import linprog, milp, LinearConstraint, Bounds

### Problem Data

In [2]:
knapsack_capacity = 5

items = {
    'Item Id': [1, 2, 3, 4, 5, 6, 7],
    'Item Name': [
        'Compass',
        'Knife',
        'Fire Starter Kit',
        'Tent',
        'Propane Cylinder',
        'Food',
        'Portable Telescope'
    ],
    'Weight (kg)': [0.5, 1.5, 0.4, 1.0, 1.6, 1.1, 0.8],
    'Relative Value': [6, 5, 4, 4, 3, 4, 1]
}

df = pd.DataFrame(items)
print(df)

   Item Id           Item Name  Weight (kg)  Relative Value
0        1             Compass          0.5               6
1        2               Knife          1.5               5
2        3    Fire Starter Kit          0.4               4
3        4                Tent          1.0               4
4        5    Propane Cylinder          1.6               3
5        6                Food          1.1               4
6        7  Portable Telescope          0.8               1


### Problem Definition

**Objective:**  
$
\max \sum \limits_{i=1}^{n} x_i \times V_i
$

**Subject to constraints:**  
$
\sum \limits_{i=1}^{n} x_i \times w_i  <= C
$

**Where:**   
$C$ is the total knapsack capacity  
$x_i \in \{0,1\}$ i.e. $x_i$ is a binary decision variable for each item available for selection (0 indicates do not select and 1 indicates selection)  
$V_i$ is the relative value of each item available for selection  
$w_i$ is the weight of each item that is available for selection


### Scipy Solution

**Library Documentation:** https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.milp.html

Define Problem Variables

In [3]:
# Note: Scipy has only minimization available. Hence, we are multiplying the objective by -1 to turn it into a maximization problem
decision_variables = -np.array([1] * 7) * np.array(items['Relative Value'])
print("Decision Variables:", decision_variables)

lower_bounds = [0 for i in range(0, len(decision_variables))]
upper_bounds = [1 for i in range(0, len(decision_variables))]
print("Decision Variables Lower Bounds:", lower_bounds)
print("Decision Variables Lower Bounds:", upper_bounds)

# From the docs, 1 : Integer variable; decision variable must be an integer within bounds.
integrality = [1] * len(decision_variables)
print("Integrality: ", integrality)

bounds = Bounds(lb = lower_bounds, ub = upper_bounds)

constraints = LinearConstraint(items['Weight (kg)'], lb = 0, ub = knapsack_capacity)

Decision Variables: [-6 -5 -4 -4 -3 -4 -1]
Decision Variables Lower Bounds: [0, 0, 0, 0, 0, 0, 0]
Decision Variables Lower Bounds: [1, 1, 1, 1, 1, 1, 1]
Integrality:  [1, 1, 1, 1, 1, 1, 1]


Solve Model

In [4]:
sol = milp(
    c = decision_variables,
    bounds = bounds,
    constraints = constraints,
    integrality = integrality
)

print(sol)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -23.0
              x: [ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  0.000e+00
                   1.000e+00  0.000e+00]
 mip_node_count: 1
 mip_dual_bound: -23.0
        mip_gap: 0.0


Model Results

In [5]:
print("Decision Variables:", sol.x)

print("Objective Value:", sol.fun)

print("Final Knapsack Weight:", sum(sol.x * np.array(items['Weight (kg)'])))

Decision Variables: [1. 1. 1. 1. 0. 1. 0.]
Objective Value: -23.0
Final Knapsack Weight: 4.5


Final Result

In [6]:
df['scipy_items_to_select'] = [True if i==1 else False for i in sol.x]
df

,Item Id,Item Name,Weight (kg),Relative Value,scipy_items_to_select
0,1,Compass,0.5,6,True
1,2,Knife,1.5,5,True
2,3,Fire Starter Kit,0.4,4,True
3,4,Tent,1.0,4,True
4,5,Propane Cylinder,1.6,3,False
5,6,Food,1.1,4,True
6,7,Portable Telescope,0.8,1,False


### PuLP Solution

Define Problem Variables

In [7]:
# defining the problem
prob = pulp.LpProblem('knapsack_pulp', LpMaximize)

# defining the binary decision variables
x1_decision_var = LpVariable.dicts(
    name = 'x1',
    indices = items['Item Name'],
    lowBound = 0,
    upBound = 1,
    cat = LpBinary
)

weights = dict(zip(items['Item Name'], items['Weight (kg)']))
relative_values = dict(zip(items['Item Name'], items['Relative Value']))

# defining the model objective function
prob += lpSum([relative_values[i] * x1_decision_var[i] for i in items['Item Name']])

# defining the problem constraints
prob += lpSum([weights[i] * x1_decision_var[i] for i in items['Item Name']]) <= knapsack_capacity

Solve Model

In [8]:
solver = get_solver(solver = 'PULP_CBC_CMD')
results = prob.solve(solver = solver)

Model Results

In [9]:
print("Solution Status:", LpStatus[results], prob.sol_status)

print("Final Objective Value:", prob.objective.value())

pulp_decision_var_results = [x1_decision_var[i].value() for i in x1_decision_var.keys()]
print(pulp_decision_var_results)

Solution Status: Optimal 1
Final Objective Value: 23.0
[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0]


Final Results

In [10]:
df['pulp_items_to_select'] = [True if i==1 else False for i in pulp_decision_var_results]
df

,Item Id,Item Name,Weight (kg),Relative Value,scipy_items_to_select,pulp_items_to_select
0,1,Compass,0.5,6,True,True
1,2,Knife,1.5,5,True,True
2,3,Fire Starter Kit,0.4,4,True,True
3,4,Tent,1.0,4,True,True
4,5,Propane Cylinder,1.6,3,False,False
5,6,Food,1.1,4,True,True
6,7,Portable Telescope,0.8,1,False,False
